In [2]:
## all imports
from IPython.display import HTML
import numpy as np
from urllib import request # urllib2 isn't supported by P3 - https://stackoverflow.com/questions/58794540/no-module-named-urllib2-how-do-i-use-it-in-python-so-i-can-make-a-request
import bs4 #this is beautiful soup
import time
import operator
import socket
import pickle # was CPickle, but again use pickly for P3
import re # regular expressions
import csv
from datetime import date as d

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from secrets import *

# Data Scraper
#### I want to scrape Rightmove for potential properties for me and my mates to move into once this whole COVID thing is over!
I'll start simple and probably build this out over time!

@maksimKorzh's scraper here: https://github.com/maksimKorzh/one-time-scrapers/blob/master/scrapers/rightmove_scraper.py was a huge help

In [3]:
url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1400&minBedrooms=4&maxPrice=6000&minPrice=3000&radius=1.0&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='
source = request.urlopen(url).read().decode()
soup = bs4.BeautifulSoup(source, 'lxml')
pretty_soup = soup.prettify()

So now the html has been parsed to lxml, I need to create the different component variables that I'm looking for:

In [5]:
# get the headline (i.e. the number of rooms and type of building):
title = [title.text.strip() for title in soup.findAll('h2', {'class': 'propertyCard-title'})] # list comprehension woop! So I believe that the text attribute is specific to BS?
print(title)
print()

# get the address:
address = [address['content'] for address in soup.findAll('meta', {'itemprop': 'streetAddress'})] #need only the content element
print(address)
print()

#get the number of miles from the station in question:
miles = [miles.text.strip() for miles in soup.findAll('div', {'class': 'propertyCard-distance'})]
print(miles)
print()

#get the property description:
desc = [desc.text.strip() for desc in soup.findAll('span', {'itemprop':'description'})]
print(desc)
print()

#get the date added to the site:
dateadded = [dateadded.text.strip() for dateadded in soup.findAll('span', {'class': 'propertyCard-branchSummary-addedOrReduced'})]
print(dateadded)
print()

#get the estate agent's name:
agent = [agent.text.strip() for agent in soup.findAll('span', {'class': 'propertyCard-branchSummary-branchName'})]
print(agent)
print()

#get the agent's number:
number = [number.text.strip() for number in soup.findAll('a', {'class': 'propertyCard-contactsPhoneNumber'})]
print(number)
print()

#get the property's price:
price = [price.text.strip() for price in soup.findAll('span', {'class': 'propertyCard-priceValue'})]
print(price)
print()

#link to the property's page:
link = ['https://www.rightmove.co.uk' + link['href'] for link in soup.findAll('a', {'class': 'propertyCard-headerLink'})]
print(link)

['4 bedroom terraced house', '4 bedroom terraced house', '4 bedroom maisonette', '5 bedroom flat', '4 bedroom mews house', '4 bedroom house', '4 bedroom semi-detached house', '4 bedroom flat', '5 bedroom flat', '4 bedroom terraced house', '4 bedroom terraced house', '4 bedroom terraced house', '4 bedroom flat', '4 bedroom terraced house', '4 bedroom character property', '4 bedroom house', '4 bedroom terraced house', '4 bedroom house', '4 bedroom terraced house', '4 bedroom mews house', '4 bedroom house', '4 bedroom detached house', 'Property', 'Property', 'Property']

['Stmathews Rd, Brixton, SW2', "St Matthew's Road, Brixton Hill, London", 'Burgate Court, London', 'Stockwell Green', 'Hazlewood Mews, London, SW9', 'Park Hill Clapham SW4', 'Kings Avenue, London, SW4', 'Aytoun Road, London, SW9', 'Edithna Road ', 'Welby Street, Camberwell, London, SE5', 'Sudbourne Road, Brixton', 'Torrens Road, Brixton', 'Aristotle Road, SW4', 'Stockwell Park Road, London, SW9', 'Hinton Road, London, SE2

In [6]:
results = []
count = sum(1 for i in price if not i is '')
print(count)

for index in range(0, count):
    results.append({
        'date_added to file' : d.today().strftime("%d/%m/%Y"), #https://www.programiz.com/python-programming/online-compiler/?ref=957b4f50
        'title' : title[index],
        'address' : address[index],
        'miles from station' : miles[index],
        'description' : desc[index],
        'price' : price[index],
        'date added/price changed' : dateadded[index],
        'agent' : agent[index],
        'agent phone number' : number[index],
        'link to property' : link[index]
    })

22


In [7]:
print(len(results))
print()

# produces dictionary within a list

22



In [8]:
df = pd.DataFrame.from_dict(results)
df.to_csv('properties.csv',encoding='utf-8', mode='w')
df.to_csv('properties_cumulative.csv',encoding='utf-8', mode='a')